In [1]:
# First we will import neccessary libraries
import pandas as pd
import numpy as np
import tensorflow as tf
import os
import cv2 as cv
from tensorflow import keras
from sklearn.model_selection import train_test_split
from sklearn import preprocessing
import matplotlib.pyplot as plt

## to get a dictionary of key as labels and values as paths

In [2]:
path = "/kaggle/input/agricultural-crops-image-classification/Agricultural-crops"
key_value= dict()
for i in os.listdir(path):
    key_value[i] = os.listdir(path+'/'+i)

## then we will convert this dictionary into a DataFrame

In [3]:
df = []
for key in key_value.keys():
    for j in key_value[key]:
        df.append([key,path+'/'+key+'/'+j])
df1 = pd.DataFrame(df,columns=('label','path'))
df1

,label,path
0,tomato,/kaggle/input/agricultural-crops-image-classif...
1,tomato,/kaggle/input/agricultural-crops-image-classif...
2,tomato,/kaggle/input/agricultural-crops-image-classif...
3,tomato,/kaggle/input/agricultural-crops-image-classif...
4,tomato,/kaggle/input/agricultural-crops-image-classif...
...,...,...
824,Coffee-plant,/kaggle/input/agricultural-crops-image-classif...
825,Coffee-plant,/kaggle/input/agricultural-crops-image-classif...
826,Coffee-plant,/kaggle/input/agricultural-crops-image-classif...
827,Coffee-plant,/kaggle/input/agricultural-crops-image-classif...


## create a function to read images

In [4]:
def readimg(path):
    img= cv.imread(path)
    img = cv.resize(img,(224,224))
    return img

## to read images fastly we will use multiprocessing

In [5]:
import multiprocessing

# Create a Pool with 4 worker processes
with multiprocessing.Pool(4) as p:
    # Read in all of the images in parallel
    images = p.map(readimg, df1['path'])


## to encode the target labels into numerical data we will use labelencoder

In [6]:
images=np.array(images, dtype='float')
# label encoder from sklearn
le = preprocessing.LabelEncoder()
y_labels = le.fit_transform(df1['label'])

In [7]:
# to split the data into training and testing data
train_images,test_images,train_labels,test_labels = train_test_split(images,y_labels,test_size=0.25, random_state=42)

 ## defining model i.e ResNet50 as source model and two layers

In [8]:
## implementing trensfer learning
base_model = keras.applications.ResNet50(weights='imagenet', include_top=False)

# freezing the base model so we donot have to train it again
base_model.trainable = False


model = keras.Sequential([
  base_model,
  keras.layers.GlobalAveragePooling2D(),
  keras.layers.Dense(1024, activation='relu'),
  keras.layers.Dense(30, activation='softmax')
])
model.compile(optimizer='adam', loss='SparseCategoricalCrossentropy', metrics=['accuracy'])

2023-01-07 13:58:32.367682: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-01-07 13:58:32.466611: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-01-07 13:58:32.467461: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-01-07 13:58:32.469585: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compil

94781440/94765736 [==============================] - 0s 0us/step


In [9]:
model.fit(train_images,train_labels, epochs=10, batch_size=32,validation_data=(test_images, test_labels))

2023-01-07 13:58:38.209977: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:185] None of the MLIR Optimization Passes are enabled (registered 2)


Epoch 1/10


2023-01-07 13:58:41.785174: I tensorflow/stream_executor/cuda/cuda_dnn.cc:369] Loaded cuDNN version 8005


20/20 [==============================] - 12s 155ms/step - loss: 2.4691 - accuracy: 0.3849 - val_loss: 1.2747 - val_accuracy: 0.6635
Epoch 2/10
20/20 [==============================] - 2s 78ms/step - loss: 0.4803 - accuracy: 0.8615 - val_loss: 0.8384 - val_accuracy: 0.7740
Epoch 3/10
20/20 [==============================] - 2s 78ms/step - loss: 0.1589 - accuracy: 0.9581 - val_loss: 0.8015 - val_accuracy: 0.7740
Epoch 4/10
20/20 [==============================] - 2s 80ms/step - loss: 0.0615 - accuracy: 0.9936 - val_loss: 0.6740 - val_accuracy: 0.8269
Epoch 5/10
20/20 [==============================] - 2s 79ms/step - loss: 0.0334 - accuracy: 0.9968 - val_loss: 0.6669 - val_accuracy: 0.8173
Epoch 6/10
20/20 [==============================] - 2s 79ms/step - loss: 0.0204 - accuracy: 0.9968 - val_loss: 0.6834 - val_accuracy: 0.8029
Epoch 7/10
20/20 [==============================] - 2s 77ms/step - loss: 0.0104 - accuracy: 1.0000 - val_loss: 0.6570 - val_accuracy: 0.8173
Epoch 8/10
20/20 [====

# To check accuracy of model

In [10]:
from sklearn.metrics import accuracy_score

# Make predictions on the test data
predictions = model.predict(test_images)

# Convert the predictions to the class labels
predicted_labels = np.argmax(predictions, axis=1)

# Compute the accuracy score
accuracy = accuracy_score(test_labels, predicted_labels)

print('Accuracy:', accuracy)

Accuracy: 0.8269230769230769


In [11]:
from sklearn.metrics import precision_score

precision = precision_score(test_labels, predicted_labels, average='micro')

print('Precision:', precision*100)

Precision: 82.6923076923077
